In [1]:
%reload_ext autoreload

import os
import sys

# Append python path - needed to import text_recognizer
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import glob

import torch
import pytorch_lightning
import wandb

from text_recognizer.data.base_data_module import DEFAULT_NUM_WORKERS

/Users/niall.turbitt/opt/anaconda3/envs/text-recognizer/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# make it easier to separate these from training runs
%env WANDB_JOB_TYPE=profile

batch_size = 16
num_workers = DEFAULT_NUM_WORKERS  # change this number later and see how the results change

env: WANDB_JOB_TYPE=profile


In [4]:
%run ../training/run_experiment.py --wandb --profile \
  --max_epochs=1 \
  --num_sanity_val_steps=0 --limit_val_batches=0 --limit_test_batches=0 \
  --model_class=LineCNNTransformer --data_class=IAMParagraphs --loss=transformer \
  --batch_size={batch_size} --num_workers={num_workers} --precision=16 


Missing logger folder: training/logs/lightning_logs
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: niall-turbitt. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/niall.turbitt/opt/anaconda3/envs/text-recognizer/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:702: UserWarning: You passed `Trainer(accelerator='cpu', precision=16)` but native AMP is not supported on CPU. Using `precision='bf16'` instead.
  rank_zero_warn(
Using bfloat16 Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
IAMParagraphs.setup(fit): Loading IAM paragraph regions and lines...


Namespace(logger=True, enable_checkpointing=True, default_root_dir=None, gradient_clip_val=None, gradient_clip_algorithm=None, num_nodes=1, num_processes=None, devices=None, gpus=None, auto_select_gpus=False, tpu_cores=None, ipus=None, enable_progress_bar=True, overfit_batches=0.0, track_grad_norm=-1, check_val_every_n_epoch=1, fast_dev_run=False, accumulate_grad_batches=None, max_epochs=1, min_epochs=None, max_steps=-1, min_steps=None, max_time=None, limit_train_batches=None, limit_val_batches=0, limit_test_batches=0, limit_predict_batches=None, val_check_interval=None, log_every_n_steps=50, accelerator=None, strategy=None, sync_batchnorm=False, precision=16, enable_model_summary=True, num_sanity_val_steps=0, resume_from_checkpoint=None, profiler=None, benchmark=None, deterministic=None, reload_dataloaders_every_n_epochs=0, auto_lr_find=False, replace_sampler_ddp=True, detect_anomaly=False, auto_scale_batch_size=False, plugins=None, amp_backend='native', amp_level=None, move_metrics_t


   | Name                      | Type               | Params
------------------------------------------------------------------
0  | model                     | LineCNNTransformer | 12.8 M
1  | model.line_cnn            | LineCNN            | 10.1 M
2  | model.embedding           | Embedding          | 21.5 K
3  | model.fc                  | Linear             | 21.6 K
4  | model.pos_encoder         | PositionalEncoding | 0     
5  | model.transformer_decoder | TransformerDecoder | 2.6 M 
6  | train_acc                 | MulticlassAccuracy | 0     
7  | val_acc                   | MulticlassAccuracy | 0     
8  | test_acc                  | MulticlassAccuracy | 0     
9  | val_cer                   | CharacterErrorRate | 0     
10 | test_cer                  | CharacterErrorRate | 0     
11 | loss_fn                   | CrossEntropyLoss   | 0     
------------------------------------------------------------------
12.8 M    Trainable params
0         Non-trainable params
12.8 M    Tota

Model State Dict Disk Size: 51.31 MB
Epoch 0:   3%|▎         | 2/68 [21:31<11:50:19, 645.75s/it, loss=4.27, v_num=9x3x]

STAGE:2023-01-05 11:04:08 21525:3295790 ActivityProfilerController.cpp:294] Completed Stage: Warm Up


Epoch 0:   9%|▉         | 6/68 [45:04<7:45:50, 450.82s/it, loss=3.76, v_num=9x3x] 

STAGE:2023-01-05 11:27:44 21525:3295790 ActivityProfilerController.cpp:300] Completed Stage: Collection
[W collection.cpp:436] Warning: Optimizer.step#Adam.step (function operator())
[W collection.cpp:436] Warning: [pl][profile][LightningModule]TransformerLitModel.optimizer_step (function operator())
STAGE:2023-01-05 11:27:52 21525:3295790 output_json.cpp:417] Completed Stage: Post Processing


Epoch 0:  88%|████████▊ | 60/68 [7:56:30<1:03:32, 476.51s/it, loss=3.24, v_num=9x3x]

/Users/niall.turbitt/opt/anaconda3/envs/text-recognizer/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
IAMParagraphs.setup(test): Loading IAM paragraph regions and lines...


In [ ]:

latest_expt = wandb.run

try:  # add execution trace to logged and versioned binaries
    folder = wandb.run.dir
    trace_matcher = wandb.run.dir + "/*.pt.trace.json"
    trace_file = glob.glob(trace_matcher)[0]
    trace_at = wandb.Artifact(name=f"trace-{wandb.run.id}", type="trace")
    trace_at.add_file(trace_file, name="training_step.pt.trace.json")
    wandb.log_artifact(trace_at)
except IndexError:
    print("trace not found")

wandb.finish()